In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/combined-data-of-all-songs/updated_songs_data (1).csv


In [ ]:
! pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.3/169.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.4 MB/s eta 0:00:00ta 0:00:01


In [ ]:
! pip install tqdm

In [ ]:
import pandas as pd
import yt_dlp
import logging
from yt_dlp.utils import DownloadError
from tqdm import tqdm
import time
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Create directory for audio files if it doesn't exist
audio_dir = 'audio_files'
os.makedirs(audio_dir, exist_ok=True)

# Load the CSV file
df = pd.read_csv('/kaggle/input/combined-data-of-all-songs/updated_songs_data (1).csv')

# Configure yt-dlp with the proper logger
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'external_downloader_args': ['-codec:a', 'libmp3lame'],
    'quiet': True,  # Keep yt-dlp's output quiet so tqdm can display cleanly
    'noplaylist': True,
    'ignoreerrors': True,
    'logger': logger,
}

# Function to download and convert a single song
def download_and_convert_song(search_query, download_path):
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            start_time = time.time()
            ydl.download([f'ytsearch:{search_query}'])
            duration = time.time() - start_time
            logger.info(f"Downloaded and converted: {search_query} in {duration:.2f} seconds")
            return duration
        except DownloadError as e:
            logger.error(f"Download failed for {search_query}: {str(e)}")
        except Exception as e:
            logger.error(f"Unexpected error for {search_query}: {str(e)}")
    return None

# Function to manage download tasks
def download_manager(start_index, end_index):
    futures = []
    start_time = time.time()  # Start timing the download process

    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit all download tasks
        for index, row in df.iloc[start_index:end_index].iterrows():
            song_name = row['song_title']
            artist_name = row['artist_name']
            query = f"{song_name} {artist_name}"
            file_path = f"{audio_dir}/{song_name.replace('/', '')} - {artist_name.replace('/', '')}.mp3"
            futures.append(executor.submit(download_and_convert_song, query, file_path))

        # Use tqdm to display progress
        for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading songs", unit="song"):
            future.result()

    elapsed_time = time.time() - start_time  # Calculate total elapsed time
    print(f"Total elapsed time for all downloads: {elapsed_time:.2f} seconds")

# Define starting and ending points for the download directly in the script
start_index = 0
end_index = 3000  # Adjust as needed

download_manager(start_index, end_index)
print("All downloads attempted.")


KeyboardInterrupt

